In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import sys
import time
from os import listdir
from os.path import isfile, join

import collections
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from scipy import misc
import matplotlib.pyplot as plt
%matplotlib inline

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
WORK_DIRECTORY = '.'
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 100
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 37
VALIDATION_SIZE = 100  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 1
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 1
EVAL_FREQUENCY = 100  # Number of steps between evaluations.


FLAGS = None

In [2]:
def data_type():
    return tf.float32

In [3]:
def maybe_download(filename):
    """Download the data from Yann's website, unless it's already here."""
    if not tf.gfile.Exists(WORK_DIRECTORY):
        tf.gfile.MakeDirs(WORK_DIRECTORY)
    filepath = os.path.join(WORK_DIRECTORY, filename)
    if not tf.gfile.Exists(filepath):
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
        size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

In [4]:
def extract_data(filename):
    """Extract the images into a 4D tensor [image index, y, x, channels].

    //Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    print('Extracting', filename)
    '''
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
        data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
        data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
        data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    '''
    files = [f for f in listdir(filename)]
    data = dict()
    for file in files:
        #f = misc.face(gray=True)
        image = misc.imread(filename+file,flatten=True)
        image.resize((32,100,))
        image = (image - image.mean())/ image.std()
        #plt.figure()
        #plt.imshow(image) 
        data[file] = image
    od = collections.OrderedDict(sorted(data.items()))
    #print(od.keys())
    return od.values()

In [5]:
import string
transductor = dict()
for i in range(10):
    transductor[str(i)] = i
for letter in  list(string.ascii_lowercase):
    transductor[letter] = ord(letter) - ord('a') + 10


In [6]:
def convert(str):
    """
        Convert a string into an array of ints
    """
    arr = np.zeros((len(str),37))
    i = 0
    for char in str:
        if char in transductor.keys():
            arr[i][transductor[char]] = 1
        else:
            arr[i][36] = 1
        i += 1
    return (arr)

In [116]:
#convert('proper')

In [7]:
def extract_labels(filename):
    """Extract the labels from a txt file into a dictionary labels whose key is the image name, 
       whose value is the word in the image
    """
    print('Extracting', filename)
    '''
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
    return labels
    '''
    labels = dict()
    data = []
    with open(filename) as f:
        for line in f.readlines():
            line = line.strip()
            content = line.split(', ')
            filename = content[0]
            label = content[1].replace('"','')
            labels[filename] = label.lower() 
            #print(filename+','+label,len(filename),len(label))
            #labels[] = 
    od = collections.OrderedDict(sorted(labels.items()))
    for string in od.values():
        code = np.zeros((23,37))
        code[:len(string)] = convert(string)
        data.append(code)
    return data

In [1]:
#extract_data('./train/')

In [121]:
tt = extract_labels('train_labels.txt')
print(len(tt))

Extracting train_labels.txt
848


In [8]:
def error_rate(predictions, labels):
  """Return the error rate based on dense predictions and sparse labels."""
  return 100.0 - (
      100.0 *
      numpy.sum(numpy.argmax(predictions, 1) == labels) /
      predictions.shape[0])


In [9]:
# Get the data.
#train_data_filename = maybe_download('Challenge2_Training_Task3_Images_GT.zip')
#train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
#test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
#test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

train_data_filename = './train/'
train_labels_filename = 'train_labels.txt'
test_data_filename = './test/'
test_labels_filename = 'test_labels.txt'

# Extract it into numpy arrays.
temp = extract_data(train_data_filename)
train_data = np.asarray(list(temp)).reshape(len(temp), IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)
temp = extract_labels(train_labels_filename)
train_labels = np.asarray(list(temp))

test_data = np.asarray(list(extract_data(test_data_filename)))
test_labels = np.asarray(list(extract_labels(test_labels_filename)))

# Generate a validation set.
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]
num_epochs = NUM_EPOCHS
train_size = train_labels.shape[0]


Extracting ./train/
Extracting train_labels.txt
Extracting ./test/
Extracting test_labels.txt


In [10]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
  data_type(),
  shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,23,))
eval_data = tf.placeholder(
  data_type(),
  shape=(EVAL_BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS))

# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when we call:
# {tf.global_variables_initializer().run()}
conv1_weights = tf.Variable(
  tf.truncated_normal([5, 5, NUM_CHANNELS, 64],  # 5x5 filter, depth 64.
                      stddev=0.1,
                      seed=SEED, dtype=data_type()))
conv1_biases = tf.Variable(tf.zeros([64], dtype=data_type()))
conv3_weights = tf.Variable(tf.truncated_normal(
  [5, 5, 64, 128], stddev=0.1,
  seed=SEED, dtype=data_type()))
conv3_biases = tf.Variable(tf.constant(0.1, shape=[128], dtype=data_type()))

conv5_weights = tf.Variable(tf.truncated_normal(
  [3, 3,128, 256], stddev=0.1,
  seed=SEED, dtype=data_type()))
conv5_biases = tf.Variable(tf.constant(0.1, shape=[256], dtype=data_type()))

conv7_weights = tf.Variable(tf.truncated_normal(
  [3, 3, 256, 512], stddev=0.1,
  seed=SEED, dtype=data_type()))
conv7_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=data_type()))

conv9_weights = tf.Variable(tf.truncated_normal(
  [3, 3, 512, 512], stddev=0.1,
  seed=SEED, dtype=data_type()))
conv9_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=data_type()))


fc6_weights = tf.Variable(  # fully connected, depth 4096.
  tf.truncated_normal([512*4*13, 4096],
                      stddev=0.1,
                      seed=SEED,
                      dtype=data_type()))
fc6_biases = tf.Variable(tf.constant(0.1, shape=[4096], dtype=data_type()))
fc7_weights = tf.Variable(tf.truncated_normal([4096,37],
                                            stddev=0.1,
                                            seed=SEED,
                                            dtype=data_type()))
fc7_biases = tf.Variable(tf.constant(
  0.1, shape=[37], dtype=data_type()))



In [69]:
print(fc7_weights)

Tensor("Variable_101/read:0", shape=(4096, 37), dtype=float32)


## We will replicate the model structure for the training subgraph, as well as the evaluation subgraphs, while sharing the trainable parameters.

In [11]:

conv1 = []

class Model():
        
    def model(self, data, train=False):
        """The Model definition."""
        # 2D convolution, with 'SAME' padding (i.e. the output feature map has
        # the same size as the input). Note that {strides} is a 4D array whose
        # shape matches the data layout: [image index, y, x, depth].
        self.conv1 = tf.nn.conv2d(data,
                            conv1_weights,
                            strides=[1, 1, 1, 1],
                            padding='SAME')
        # Bias and rectified linear non-linearity.
        self.relu1 = tf.nn.relu(tf.nn.bias_add(self.conv1, conv1_biases))
        # Max pooling. The kernel size spec {ksize} also follows the layout of
        # the data. Here we have a pooling window of 2, and a stride of 2.
        self.pool2 = tf.nn.max_pool(self.relu1,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')
        
        self.conv3 = tf.nn.conv2d(self.pool2,
                            conv3_weights,
                            strides=[1, 1, 1, 1],
                            padding='SAME')
        self.relu3 = tf.nn.relu(tf.nn.bias_add(self.conv3, conv3_biases))
        self.pool4 = tf.nn.max_pool(self.relu3,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')
        
        self.conv5 = tf.nn.conv2d(self.pool4,
                            conv5_weights,
                            strides=[1, 1, 1, 1],
                            padding='SAME')
        self.relu5 = tf.nn.relu(tf.nn.bias_add(self.conv5, conv5_biases))
        self.pool5 = tf.nn.max_pool(self.relu5,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')
        self.conv7 = tf.nn.conv2d(self.pool5,
                            conv7_weights,
                            strides=[1, 1, 1, 1],
                            padding='SAME')
        self.relu7 = tf.nn.relu(tf.nn.bias_add(self.conv7, conv7_biases))
        
        self.conv9 = tf.nn.conv2d(self.relu7,
                            conv9_weights,
                            strides=[1, 1, 1, 1],
                            padding='SAME')
        self.relu9 = tf.nn.relu(tf.nn.bias_add(self.conv9, conv9_biases))
        
        # Reshape the feature map cuboid into a 2D matrix to feed it to the
        # fully connected layers.
        conv_shape = self.relu9.get_shape().as_list()
        
        reshape = tf.reshape(
            self.relu9,
            [conv_shape[0], conv_shape[1] * conv_shape[2] * conv_shape[3]])
        # Fully connected layer. Note that the '+' operation automatically
        # broadcasts the biases.
        self.hidden = tf.nn.relu(tf.matmul(reshape, fc6_weights) + fc6_biases)
        # Add a 50% dropout during training only. Dropout also scales
        # activations such that no rescaling is needed at evaluation time.
        if train:
            self.hidden = tf.nn.dropout(self.hidden, 0.5, seed=SEED)
        self.outputs = []
        for i in range(23):
            self.outputs.append(tf.matmul(self.hidden, fc7_weights)+fc7_biases)
        self.outputs = tf.stack(self.outputs, axis=0)
        print(self.outputs)
        return self.outputs


In [12]:
# Training computation: logits + cross-entropy loss.
model_train = Model()
logits = model_train.model(train_data_node, True)
print(logits)
print(train_labels_node)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=train_labels_node))

# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(conv9_weights) + tf.nn.l2_loss(conv9_biases) + tf.nn.l2_loss(fc6_weights) + tf.nn.l2_loss(fc6_biases) +
              tf.nn.l2_loss(fc7_weights) + tf.nn.l2_loss(fc7_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=data_type())
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
  0.01,                # Base learning rate.
  batch * BATCH_SIZE,  # Current index into the dataset.
  train_size,          # Decay step.
  0.95,                # Decay rate.
  staircase=True)
# Use simple momentum for the optimization.
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                     0.9).minimize(loss,
                                                   global_step=batch)

# Predictions for the current training minibatch.
train_prediction = tf.nn.softmax(logits)

model_eval = Model()
# Predictions for the test and validation, which we'll compute less often.
eval_prediction = tf.nn.softmax(model_eval.model(eval_data))


Tensor("stack:0", shape=(23, 1, 37), dtype=float32)
Tensor("stack:0", shape=(23, 1, 37), dtype=float32)
Tensor("Placeholder_1:0", shape=(1, 23), dtype=int64)
Tensor("stack_1:0", shape=(23, 1, 37), dtype=float32)


In [76]:
print(logits)

Tensor("stack_19:0", shape=(23, 1, 37), dtype=float32)


In [77]:
print(train_prediction.values())

AttributeError: 'Tensor' object has no attribute 'values'

In [13]:
# Small utility function to evaluate a dataset by feeding batches of data to
# {eval_data} and pulling the results from {eval_predictions}.
# Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
        raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
        else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
        predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions

# Create a local session to run the training.

In [14]:
import numpy as np
start_time = time.time()
sess = tf.Session()

# Run all the initializers to prepare the trainable parameters.
tf.global_variables_initializer().run(session=sess)
print('Initialized!')
# Loop through training steps.
for step in xrange(int(num_epochs * train_size) // BATCH_SIZE):
    # Compute the offset of the current minibatch in the data.
    # Note that we could use better randomization across epochs.
    offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
    batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    #print(batch_labels.shape())
    print(train_labels_node[0])
    # This dictionary maps the batch data (as a numpy array) to the
    # node in the graph it should be fed to.
    feed_dict = {train_data_node: batch_data,
               train_labels_node: batch_labels}
    # Run the optimizer to update weights.
    sess.run(optimizer, feed_dict=feed_dict)
    # print some extra information once reach the evaluation frequency
    if step % EVAL_FREQUENCY == 0:
        # fetch some extra nodes' data
        l, lr, predictions = sess.run([loss, learning_rate, train_prediction],
                                      feed_dict=feed_dict)
        elapsed_time = time.time() - start_time
        start_time = time.time()
        
        #getActivations(conv1,batch_data[0])
        print('Step %d (epoch %.2f), %.1f ms' %
              (step, float(step) * BATCH_SIZE / train_size,
               1000 * elapsed_time / EVAL_FREQUENCY))
        print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
        print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
        print('Validation error: %.1f%%' % error_rate(
            eval_in_batches(validation_data, sess), validation_labels))
        sys.stdout.flush()
# Finally print the result!
test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
print('Test error: %.1f%%' % test_error)

Initialized!
Tensor("strided_slice:0", shape=(23,), dtype=int64)


ValueError: Cannot feed value of shape (1, 23, 37) for Tensor 'Placeholder_1:0', which has shape '(1, 23)'

In [103]:
print(len(batch_labels))

1


In [ ]:
def getActivations(layer,stimuli):
        units = sess.run(layer,feed_dict={train_data_node:np.reshape(stimuli,[64,28,28,1],order='F')})
        plotNNFilter(units)

In [ ]:
import math

def plotNNFilter(units):
    filters = units.shape[3]
    print (units.shape)
    plt.figure(1, figsize=(20,20))
    n_columns = 6
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="gray")

In [ ]:
imageToUse = batch_data[0:64]
print (len(imageToUse))
plt.imshow(np.reshape(imageToUse[0],[28,28]), interpolation="nearest", cmap="gray")

# Convolutional Layer C1

In [ ]:
getActivations(model_train.conv1,imageToUse)

# ReLU Layer Relu1

In [ ]:
getActivations(model_train.relu1,imageToUse)

# Pool Layer S2

In [ ]:
getActivations(model_train.pool2,imageToUse)

# Convolutional Layer C3

In [ ]:
getActivations(model_train.conv3,imageToUse)

# ReLU Layer relu3

In [ ]:
getActivations(model_train.relu3,imageToUse)

# Pool Layer S4

In [ ]:
getActivations(model_train.pool4,imageToUse)